# Notebook to run SMF policy and SARSOP policy

## Load packages

In [1]:
using Printf
using Distributed
using Statistics
using CPUTime
using StatsBase
using Random

include("src/parser.jl")
include("src/simulations.jl")
include("src/models/model_MDP.jl")

Random.seed!(0)

## Enter parameters:
### - filename: instance name
### - nb_sims: number of simulations
### - Horizon: horizon time 
### - T: horizon time of the MILP
### - T_bound: horizon time to compute the value of the upper bound $\tilde{z}_{\mathrm{R}^\mathrm{c}}^{T_{bound}}$

In [5]:
# define the instance
filename = "tiger"

# parameters
nb_sims = 100
Horizon = 500
T = 5
T_bound = 100;

## Read the instance and load the parameters in variables

In [2]:
################Read the instance and load it##############################################
S, O, A, p_init,p_emis, p_trans, reward = parser_POMDP(string("instances/",filename,".dat"))

# compute the initial belief state
p_init = p_init./sum(p_init)
p_trans = p_trans./sum(p_trans,dims=3)
p_emis = p_emis./sum(p_emis,dims=3)

discount=0.95


nbS = length(S)
nbO = length(O)
nbA = length(A)

## Compute valid cuts probability distribution using Bayes formula
## $p(s'|s,a,o)$

In [6]:
p_cuts = zeros(length(S),length(A),length(S), length(O))

for s in S, a in A, ss in S, oo in O
    if sum(p_trans[s,a,sss]*p_emis[a,sss,oo] for sss in S) == 0
        if ss == 1
                p_cuts[s,a,ss,oo] = 1.0
        else
                p_cuts[s,a,ss,oo] = 0.0
        end
    else
        p_cuts[s,a,ss,oo] = p_trans[s,a,ss]*p_emis[a,ss,oo]/sum(p_trans[s,a,sss]*p_emis[a,sss,oo] for sss in S)
    end
end

## Compute MDP value function as tailReward

In [3]:
############################### Compute MDP value function with value #####
tailReward = zeros(nbS)
mdp = model_MDP(S,A,p_init,p_trans,reward,discount)

optimize!(mdp)
v_function = JuMP.value.(mdp[:v_s])
for s in S
        tailReward[s] = v_function[s]
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600


## Define simulation parameters 

In [7]:
init_state = zeros(Int64, nb_sims)
init_obs = zeros(Int64, nb_sims)
belief_state = ones(Float64,nb_sims,nbS)

# the first is drwan according to uniform distribution 
p_uniform = (1/nbS)* ones(nbS)

# define the set of initial states and observations 
for sim in 1:nb_sims
        init_state[sim] = sample(S,Weights(p_uniform))
        init_obs[sim] = sample(O,Weights(p_emis[1,init_state[sim],:]))
        belief_state[sim,:] = p_init
end

## Compute upper bound $\tilde{z}_{\mathrm{R}}^T$

In [9]:
# define linear relaxation
model_lp = model_tailedMILP(T_bound,S,O,A,p_init,p_trans,p_emis,reward,tailReward,discount)
model_lp = relax_model(model_lp)
optimize!(model_lp)


# get the values
obj_lp = objective_value(model_lp)
opt_status = termination_status(model_lp)
obj_bound_lp = JuMP.objective_bound(model_lp)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001


188.9999999999998

## Compute upper bound $\tilde{z}_{\mathrm{R}^{\mathrm{c}}}^T$

In [13]:
###################### Compute upper bounds small horizon #########################
model_lp = model_tailedMILP_cuts(T_bound,S,O,A,p_init,p_trans,p_emis,p_cuts,reward,tailReward,discount)
model_lp = relax_model(model_lp)
optimize!(model_lp)


obj_lp_cuts = objective_value(model_lp_cuts)
opt_cuts_status = termination_status(model_lp_cuts)

if opt_cuts_status== "TIME_LIMIT"
        obj_lp_cuts = JuMP.objective_bound(model_lp_cuts)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001


## Compute SARSOP policy

In [15]:
#######################################################################################################
@printf "=============== POMDP policies on discounted infinite horizon =======\n"
pomdp = def_pomdp(S,O,A,p_init,p_emis,p_trans,reward,discount)
start = time()
SARSOPpolicy = compute_policy(filename,pomdp)
SARSOP_time = time()-start
#######################################################################################################

=============== POMDP policies on discounted infinite horizon =======
Generating a pomdpx file: tiger.pomdpx

Loading the model ...
  input file   : tiger.pomdpx
  loading time : 0.00s 

SARSOP initializing ...
  initialization time : 0.00s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0       0       0        -20        92.8206    112.821     3        1        
 0.01    2       51       -6.2981    63.1396    69.4377     7        16       
 0.01    4       103      0.149651   52.2764    52.1268     9        21       
 0.01    6       151      6.19248    42.0546    35.8621     9        21       
 0.01    8       200      10.3563    35.232     24.8757     12       21       
 0.01    11      250      14.0433    29.5471    15.5037     6        21       
 0.01    14      300      16.545     25.0926

0.0388028621673584

## Run simulations

In [18]:
# Rewards of SMF policy
SMF_reward = zeros(nb_sims)
SMF_time = zeros(nb_sims)

# Reward of SARSOP policy
SARSOP_reward = zeros(nb_sims)
SARSOP_time = SARSOP_time/nb_sims

for sim in 1:nb_sims
    @printf "================Start simulation %.0f ============================\n" sim

    ###########################################################

    Random.seed!(sim)
    @printf "=============Run SMF policy T=%.0f ================== \n" T
    ######## Run SMF policy ########
    policy_reward, policy_time = simulation("SMF",SARSOPpolicy,Horizon,T,S,O,A,p_init,p_emis,p_trans,reward,tailReward,discount,init_state[sim],init_obs[sim])
    SMF_reward[sim] = policy_reward
    SMF_time[sim] = policy_time
    #####################################################

    Random.seed!(sim)
    @printf "=============Run SARSOP policy ================== \n"
    
    ########Run MILP heuristic with cuts########
    policy_reward, policy_time = simulation("SARSOP",SARSOPpolicy,Horizon,T,S,O,A,p_init,p_emis,p_trans,reward,tailReward,discount,init_state[sim],init_obs[sim])
    SARSOP_reward[sim] = policy_reward
    #####################################################

end

================Start simulation 1 ============================
=============Run SMF policy T=5 ================== 
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.001
Set parameter Username
Academic license - for non-commercia

## Compute the optimality gap $G^{T_{bound}}$

In [19]:
best_bound = [min(obj_lp_cuts,obj_lp) for i in 1:nb_sims]

# compute gaps on larger horizon (smaller gaps)
SMF_gap = 100*(best_bound - SMF_reward)./abs.(best_bound)
SARSOP_gap = 100*(best_bound - SARSOP_reward)./abs.(best_bound)

2-element Vector{Float64}:
 91.84023005847688
 86.12216627272713

## Display the results

In [23]:
@printf "============================================================================\n"
@printf "===================================================================\n"
@printf "===========================================================\n"
@printf "==============================================\n"
@printf "============= Results ==================\n"

@printf "Upper bound with inequalities (11) and T=T_bound : %f \n" obj_lp_cuts
@printf "Upper bound with T=T_bound : %f \n\n" obj_lp

@printf "===== SMF policy T=%.0f =====\n" T
@printf "SMF policy gap with T_bound : %f \n" mean(SMF_gap)
@printf "SMF policy reward : %f \n" mean(SMF_reward)
@printf "SMF policy time : %f \n\n" mean(SMF_time)

@printf "===== SARSOP policy T=%.0f =====\n" T
@printf "SARSOP policy gap with T_bound : %f \n" mean(SARSOP_gap)
@printf "SARSOP policy reward : %f \n" mean(SARSOP_reward)
@printf "SARSOP policy time : %f \n\n" SARSOP_time

============= Results ==================
Upper bound with inequalities (11) and T=T_bound : 127.624053 
Upper bound with T=T_bound : 189.000000 

===== SMF policy T=5 =====
SMF policy gap with T_bound : 96.542605 
SMF policy reward : 4.412468 
SMF policy time : 0.041419 

===== SARSOP policy T=5 =====
SARSOP policy gap with T_bound : 88.981198 
SARSOP policy reward : 14.062641 
SARSOP policy time : 0.019401 

